In [1]:
!pip install geopandas pyarrow shapely pyproj rtree matplotlib pandas numpy openpyxl jupyterlab seaborn plotly scipy scikit-learn pulp


In [2]:
import pandas as pd
from pathlib import Path
import numpy as np

# Creating Synthetic Fields

In [3]:
# --- CONFIG ---
N_FIELDS = 20

# Rough bounding box for central Illinois
MIN_LAT, MAX_LAT = 39.0, 41.0
MIN_LON, MAX_LON = -90.5, -87.5

rng = np.random.default_rng(42)

# 1) Sample realistic-ish field sizes (acres)
# We'll use a lognormal-like distribution and clip to [40, 120] acres
raw_acres = rng.lognormal(mean=4.0, sigma=0.35, size=N_FIELDS)  # raw in arbitrary units
scaled_acres = np.interp(raw_acres,
                         (raw_acres.min(), raw_acres.max()),
                         (40, 120))  # scale to 40–120 acre range

# 2) Sample fake centroids within IL bounding box
lats = rng.uniform(MIN_LAT, MAX_LAT, size=N_FIELDS)
lons = rng.uniform(MIN_LON, MAX_LON, size=N_FIELDS)

# 3) Build DataFrame
fields = pd.DataFrame({
    "field_id": [f"F{i+1:03d}" for i in range(N_FIELDS)],
    "crop_code": 1,           # 1 = corn, consistent with CDL/NASS
    "crop_name": "CORN",
    "acres": scaled_acres,
    "centroid_lat": lats,
    "centroid_lon": lons,
    "state_fips": "17",       # Illinois
    "state_name": "Illinois"
})

print(fields.head())
print("\nSummary of acres:")
print(fields["acres"].describe())

# 4) Save to CSV for your modeling pipeline
fields.to_csv("illinois_corn_fields_synthetic.csv", index=False)
print("\nSaved illinois_corn_fields_synthetic.csv")


  field_id  crop_code crop_name       acres  centroid_lat  centroid_lon  \
0     F001          1      CORN   89.656563     40.516175    -89.188544   
1     F002          1      CORN   55.511201     39.709052    -88.001965   
2     F003          1      CORN  105.013233     40.941396    -88.399205   
3     F004          1      CORN  112.327980     40.786242    -89.562900   
4     F005          1      CORN   40.000000     40.556767    -88.003221   

  state_fips state_name  
0         17   Illinois  
1         17   Illinois  
2         17   Illinois  
3         17   Illinois  
4         17   Illinois  

Summary of acres:
count     20.000000
mean      82.938715
std       23.572942
min       40.000000
25%       59.319511
50%       83.281591
75%      105.268789
max      120.000000
Name: acres, dtype: float64

Saved illinois_corn_fields_synthetic.csv


In [4]:
df_harvested = 'src/CORN, GRAIN – PROGRESS, MEASURED IN PCT HARVESTED.csv'
df_fields  = 'src/illinois_corn_fields_synthetic.csv'
df_noaa_d  = 'src/noaa_il_daily_raw.csv'
df_noaa_w  = 'src/noaa_il_weekly_agg.csv'
df_labor   = 'src/no_of_worker2.csv'

In [5]:
df_planting = pd.read_csv('/home/mak/Documents/Optimization/Project/data/raw/CORNPROGRESSMEASURED IN PCT PLANTED.csv')

In [6]:
df_planting.info

<bound method DataFrame.info of     Program  Year    Period Week Ending Geo Level     State  State ANSI  \
0    SURVEY  2025  WEEK #13  2025-03-30     STATE  ILLINOIS          17   
1    SURVEY  2025  WEEK #14  2025-04-06     STATE  ILLINOIS          17   
2    SURVEY  2025  WEEK #15  2025-04-13     STATE  ILLINOIS          17   
3    SURVEY  2025  WEEK #16  2025-04-20     STATE  ILLINOIS          17   
4    SURVEY  2025  WEEK #17  2025-04-27     STATE  ILLINOIS          17   
..      ...   ...       ...         ...       ...       ...         ...   
195  SURVEY  2005  WEEK #16  2005-04-24     STATE  ILLINOIS          17   
196  SURVEY  2005  WEEK #17  2005-05-01     STATE  ILLINOIS          17   
197  SURVEY  2005  WEEK #18  2005-05-08     STATE  ILLINOIS          17   
198  SURVEY  2005  WEEK #19  2005-05-15     STATE  ILLINOIS          17   
199  SURVEY  2005  WEEK #20  2005-05-22     STATE  ILLINOIS          17   

     Ag District  Ag District Code  County  ...  Zip Code  Region  

In [7]:
df_planting.dtypes


Program              object
Year                  int64
Period               object
Week Ending          object
Geo Level            object
State                object
State ANSI            int64
Ag District         float64
Ag District Code    float64
County              float64
County ANSI         float64
Zip Code            float64
Region              float64
watershed_code        int64
Watershed           float64
Commodity            object
Data Item            object
Domain               object
Domain Category      object
Value                 int64
CV (%)              float64
dtype: object

In [8]:
print("Raw shape:", df_planting.shape)
df_planting.head()

Raw shape: (200, 21)


,Program,Year,Period,Week Ending,Geo Level,State,State ANSI,Ag District,Ag District Code,County,...,Zip Code,Region,watershed_code,Watershed,Commodity,Data Item,Domain,Domain Category,Value,CV (%)
0,SURVEY,2025,WEEK #13,2025-03-30,STATE,ILLINOIS,17,NaN,NaN,NaN,...,NaN,NaN,0,NaN,CORN,"CORN - PROGRESS, MEASURED IN PCT PLANTED",TOTAL,NOT SPECIFIED,0,NaN
1,SURVEY,2025,WEEK #14,2025-04-06,STATE,ILLINOIS,17,NaN,NaN,NaN,...,NaN,NaN,0,NaN,CORN,"CORN - PROGRESS, MEASURED IN PCT PLANTED",TOTAL,NOT SPECIFIED,0,NaN
2,SURVEY,2025,WEEK #15,2025-04-13,STATE,ILLINOIS,17,NaN,NaN,NaN,...,NaN,NaN,0,NaN,CORN,"CORN - PROGRESS, MEASURED IN PCT PLANTED",TOTAL,NOT SPECIFIED,1,NaN
3,SURVEY,2025,WEEK #16,2025-04-20,STATE,ILLINOIS,17,NaN,NaN,NaN,...,NaN,NaN,0,NaN,CORN,"CORN - PROGRESS, MEASURED IN PCT PLANTED",TOTAL,NOT SPECIFIED,7,NaN
4,SURVEY,2025,WEEK #17,2025-04-27,STATE,ILLINOIS,17,NaN,NaN,NaN,...,NaN,NaN,0,NaN,CORN,"CORN - PROGRESS, MEASURED IN PCT PLANTED",TOTAL,NOT SPECIFIED,16,NaN


## Filter only Illinois & planting rows

In [9]:
df_planting = df_planting[
    (df_planting["State"] == "ILLINOIS") &
    (df_planting["Data Item"].str.contains("PCT PLANTED"))
]

In [10]:
df_planting.head(3)

,Program,Year,Period,Week Ending,Geo Level,State,State ANSI,Ag District,Ag District Code,County,...,Zip Code,Region,watershed_code,Watershed,Commodity,Data Item,Domain,Domain Category,Value,CV (%)
0,SURVEY,2025,WEEK #13,2025-03-30,STATE,ILLINOIS,17,NaN,NaN,NaN,...,NaN,NaN,0,NaN,CORN,"CORN - PROGRESS, MEASURED IN PCT PLANTED",TOTAL,NOT SPECIFIED,0,NaN
1,SURVEY,2025,WEEK #14,2025-04-06,STATE,ILLINOIS,17,NaN,NaN,NaN,...,NaN,NaN,0,NaN,CORN,"CORN - PROGRESS, MEASURED IN PCT PLANTED",TOTAL,NOT SPECIFIED,0,NaN
2,SURVEY,2025,WEEK #15,2025-04-13,STATE,ILLINOIS,17,NaN,NaN,NaN,...,NaN,NaN,0,NaN,CORN,"CORN - PROGRESS, MEASURED IN PCT PLANTED",TOTAL,NOT SPECIFIED,1,NaN


## 🔧 Step 2: Extract numeric week from Period
### Your Period looks like:
### "WEEK #13"
### "WEEK #17"
### We extract the number:

In [11]:
df_planting["week"] = df_planting["Period"].str.extract(r"(\d+)").astype(int)
# df_planting.head(3)
type(df_planting["week"][0])

numpy.int64

## 🔧 Step 3: Convert Week Ending → datetime

In [12]:
df_planting["week_ending"] = pd.to_datetime(df_planting["Week Ending"])
type(df_planting["week_ending"][0])

pandas._libs.tslibs.timestamps.Timestamp

In [13]:
df_planting.head(2)

,Program,Year,Period,Week Ending,Geo Level,State,State ANSI,Ag District,Ag District Code,County,...,watershed_code,Watershed,Commodity,Data Item,Domain,Domain Category,Value,CV (%),week,week_ending
0,SURVEY,2025,WEEK #13,2025-03-30,STATE,ILLINOIS,17,NaN,NaN,NaN,...,0,NaN,CORN,"CORN - PROGRESS, MEASURED IN PCT PLANTED",TOTAL,NOT SPECIFIED,0,NaN,13,2025-03-30
1,SURVEY,2025,WEEK #14,2025-04-06,STATE,ILLINOIS,17,NaN,NaN,NaN,...,0,NaN,CORN,"CORN - PROGRESS, MEASURED IN PCT PLANTED",TOTAL,NOT SPECIFIED,0,NaN,14,2025-04-06


## 🔧 Step 4: Rename Value → pct_planted

In [14]:
df_planting["pct_planted"] = df_planting["Value"].astype(float)
type(df_planting["pct_planted"][0])

numpy.float64

## 🔧 Step 5: Keep only the needed columns

In [15]:
df_planting_clean = df_planting[["Year", "week", "week_ending", "pct_planted"]].sort_values(["Year", "week"])
df_planting_clean.head()


,Year,week,week_ending,pct_planted
194,2005,15,2005-04-17,35.0
195,2005,16,2005-04-24,64.0
196,2005,17,2005-05-01,82.0
197,2005,18,2005-05-08,94.0
198,2005,19,2005-05-15,98.0


## 🌾 5. Compute planting windows (per-year)

### Planting window logic:
### Start: first week where pct_planted > 1
### End: first week where pct_planted ≥ 90

In [16]:
windows = []
for year, g in df_planting_clean.groupby("Year"):
    g = g.sort_values("week")
    
    start = g[g["pct_planted"] > 1]["week"].min()
    end = g[g["pct_planted"] >= 90]["week"].min()
    
    windows.append({"Year": year,
                    "plant_start_week": start,
                    "plant_end_week": end})
    
plantingWindows_df = pd.DataFrame(windows).sort_values("Year")
plantingWindows_df


,Year,plant_start_week,plant_end_week
0,2005,15,18
1,2006,15,19
2,2007,16,19
3,2008,17,22
4,2009,17,23
5,2010,15,18
6,2011,14,20
7,2012,13,19
8,2013,18,22
9,2014,16,21


## 🌽 6. Compute typical (median) planting window
## This is what the optimization model will use:

In [17]:
typical_start = int(plantingWindows_df["plant_start_week"].median())
typical_end = int(plantingWindows_df["plant_end_week"].median())

print("Typical window:", typical_start, "to", typical_end)


Typical window: 16 to 21


In [18]:
df_harvested = pd.read_csv('/home/mak/Documents/Optimization/Project/data/raw/CORN, GRAIN – PROGRESS, MEASURED IN PCT HARVESTED.csv')
print(df_harvested.shape)
df_harvested.head(3)

(240, 21)


,Program,Year,Period,Week Ending,Geo Level,State,State ANSI,Ag District,Ag District Code,County,...,Zip Code,Region,watershed_code,Watershed,Commodity,Data Item,Domain,Domain Category,Value,CV (%)
0,SURVEY,2025,WEEK #35,2025-08-31,STATE,ILLINOIS,17,NaN,NaN,NaN,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - PROGRESS, MEASURED IN PCT HARVESTED",TOTAL,NOT SPECIFIED,0,NaN
1,SURVEY,2025,WEEK #36,2025-09-07,STATE,ILLINOIS,17,NaN,NaN,NaN,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - PROGRESS, MEASURED IN PCT HARVESTED",TOTAL,NOT SPECIFIED,2,NaN
2,SURVEY,2025,WEEK #37,2025-09-14,STATE,ILLINOIS,17,NaN,NaN,NaN,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - PROGRESS, MEASURED IN PCT HARVESTED",TOTAL,NOT SPECIFIED,5,NaN


## Filter to only Illinois & “PCT HARVESTED”

In [19]:
df_harvested = df_harvested[
    (df_harvested["State"] == "ILLINOIS") &
    (df_harvested["Data Item"].str.contains("PCT HARVESTED"))
]

print("After filter:", df_harvested.shape)
df_harvested.head(3)

After filter: (240, 21)


,Program,Year,Period,Week Ending,Geo Level,State,State ANSI,Ag District,Ag District Code,County,...,Zip Code,Region,watershed_code,Watershed,Commodity,Data Item,Domain,Domain Category,Value,CV (%)
0,SURVEY,2025,WEEK #35,2025-08-31,STATE,ILLINOIS,17,NaN,NaN,NaN,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - PROGRESS, MEASURED IN PCT HARVESTED",TOTAL,NOT SPECIFIED,0,NaN
1,SURVEY,2025,WEEK #36,2025-09-07,STATE,ILLINOIS,17,NaN,NaN,NaN,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - PROGRESS, MEASURED IN PCT HARVESTED",TOTAL,NOT SPECIFIED,2,NaN
2,SURVEY,2025,WEEK #37,2025-09-14,STATE,ILLINOIS,17,NaN,NaN,NaN,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - PROGRESS, MEASURED IN PCT HARVESTED",TOTAL,NOT SPECIFIED,5,NaN


## ✅ 3. Extract week number from “Period”
### The Period column looks like "WEEK #41".
### We extract the number:

In [20]:
# ---- Extract week number from Period "WEEK #xx" ----
df_harvested["week"] = df_harvested["Period"].str.extract(r'(\d+)').astype(int)
type(df_harvested["week"][0])


numpy.int64

## ✅ 4. Convert week-ending date

In [21]:
# ---- Convert week ending to datetime ----
df_harvested["week_ending"] = pd.to_datetime(df_harvested["Week Ending"], errors='coerce')
type(df_harvested["week_ending"][0])

pandas._libs.tslibs.timestamps.Timestamp

## ✅ 5. Rename Value → pct_harvested

In [22]:
# ---- Clean Value column ----
df_harvested["pct_harvested"] = pd.to_numeric(df_harvested["Value"], errors='coerce')
type(df_harvested["pct_harvested"][0])

numpy.int64

## ✅ 6. Keep only the needed columns

In [23]:
# ---- Select only useful columns ----
df_harvested_clean = df_harvested[["Year", "week", "week_ending", "pct_harvested"]] \
    .sort_values(["Year", "week"]) \
    .reset_index(drop=True)

print(df_harvested_clean.head())

   Year  week week_ending  pct_harvested
0  2005    36  2005-09-11              6
1  2005    37  2005-09-18             13
2  2005    38  2005-09-25             30
3  2005    39  2005-10-02             42
4  2005    40  2005-10-09             58


## 🌽 8. Compute harvest windows (per-year)
### We define the harvest window using typical agricultural logic:
### Start = first week with >1% harvested
### End = first week with ≥90% harvested

In [24]:
# ---- Compute harvest windows ----
harvest_windows = []

for year, g in df_harvested_clean.groupby("Year"):
    g = g.sort_values("week")

    start = g[g["pct_harvested"] > 1]["week"].min()
    end   = g[g["pct_harvested"] >= 90]["week"].min()
    print(start, end)

    harvest_windows.append({
        "Year": year,
        "harvest_start_week": start,
        "harvest_end_week": end
    })

harvest_windows_df = pd.DataFrame(harvest_windows).sort_values("Year")
print(harvest_windows_df.head())

36 43
36 44
36 42
39 47
39 50
35 41
37 44
33 42
38 45
37 46
36 43
36 43
36 45
36 44
38 48
37 45
37 44
37 45
36 44
36 44
36 nan
   Year  harvest_start_week  harvest_end_week
0  2005                  36              43.0
1  2006                  36              44.0
2  2007                  36              42.0
3  2008                  39              47.0
4  2009                  39              50.0


In [25]:
harvest_windows_df['Year'].value_counts()

Year
2005    1
2006    1
2007    1
2008    1
2009    1
2010    1
2011    1
2012    1
2013    1
2014    1
2015    1
2016    1
2017    1
2018    1
2019    1
2020    1
2021    1
2022    1
2023    1
2024    1
2025    1
Name: count, dtype: int64

## 🌽 9. Compute the typical (median) harvest window

In [26]:
# ---- Compute typical window ----
typical_start = int(harvest_windows_df["harvest_start_week"].median())
typical_end   = int(harvest_windows_df["harvest_end_week"].median())

print("Typical harvest window:")
print(f"Start week ≈ {typical_start}")
print(f"End week   ≈ {typical_end}")

Typical harvest window:
Start week ≈ 36
End week   ≈ 44


In [27]:
df_planting_clean = df_planting_clean.dropna(subset=["week"])
df_planting_clean["week"] = df_planting_clean["week"].astype(int)

# Clean harvest NASS
df_harvested_clean = df_harvested_clean.dropna(subset=["week"])
df_harvested_clean["week"] = df_harvested_clean["week"].astype(int)


### Loading Corn fields Synthetic

In [28]:
fields  = pd.read_csv('/home/mak/Documents/Optimization/Project/data/raw/illinois_corn_fields_synthetic.csv')

In [29]:
fields.head(3)

,field_id,crop_code,crop_name,acres,centroid_lat,centroid_lon,state_fips,state_name
0,F001,1,CORN,87.227749,39.389277,-89.900275,17,Illinois
1,F002,1,CORN,54.752514,39.933442,-90.477913,17,Illinois
2,F003,1,CORN,101.833288,39.087608,-88.139227,17,Illinois


### 🎯 What you get from this cleaning
####  1. Cleaned weekly harvest table

#### Columns:
#### Year | week | week_ending | pct_harvested

### 2. Harvest window table
#### Columns:
#### Year | harvest_start_week | harvest_end_week

### 3. Typical harvest window (median across years)
#### Something like:

#### Start ≈ week 36
#### End   ≈ week 44
#### This will drive the harvest constraints in your MILP model.

In [30]:
print("Raw shape:", fields.shape)
print(fields.head())

# 1. Keep only corn rows (future-proofing, even if currently all CORN)
fields = fields[fields["crop_name"].str.upper() == "CORN"].copy()

Raw shape: (25, 8)
  field_id  crop_code crop_name       acres  centroid_lat  centroid_lon  \
0     F001          1      CORN   87.227749     39.389277    -89.900275   
1     F002          1      CORN   54.752514     39.933442    -90.477913   
2     F003          1      CORN  101.833288     39.087608    -88.139227   
3     F004          1      CORN  108.790255     39.308579    -88.505447   
4     F005          1      CORN   40.000000     40.366098    -88.384504   

   state_fips state_name  
0          17   Illinois  
1          17   Illinois  
2          17   Illinois  
3          17   Illinois  
4          17   Illinois  


In [31]:
fields.head(3)
# fields["acres"].value_counts()

,field_id,crop_code,crop_name,acres,centroid_lat,centroid_lon,state_fips,state_name
0,F001,1,CORN,87.227749,39.389277,-89.900275,17,Illinois
1,F002,1,CORN,54.752514,39.933442,-90.477913,17,Illinois
2,F003,1,CORN,101.833288,39.087608,-88.139227,17,Illinois


In [32]:
# 2. Basic sanity checks
#    - acres > 0
fields = fields[fields["acres"] > 0].copy()

# 3. Define simple regions based on latitude (south/central/north Illinois)
#    You can tweak thresholds if you like.
lat = fields["centroid_lat"]

In [33]:
def assign_region(lat_val):
    if lat_val < 39.7:
        return "South"
    elif lat_val < 40.3:
        return "Central"
    else:
        return "North"

In [34]:
fields["region"] = lat.apply(assign_region)

# 4. Select only columns we actually need for modeling
fields_clean = fields[[
    "field_id",
    "acres",
    "centroid_lat",
    "centroid_lon",
    "region"
]].sort_values("field_id").reset_index(drop=True)

print(fields_clean.head())

  field_id       acres  centroid_lat  centroid_lon   region
0     F001   87.227749     39.389277    -89.900275    South
1     F002   54.752514     39.933442    -90.477913  Central
2     F003  101.833288     39.087608    -88.139227    South
3     F004  108.790255     39.308579    -88.505447    South
4     F005   40.000000     40.366098    -88.384504    North


### 3️⃣ How this feeds each process

### Optimization (Gurobi MILP)

### field_id → set of decision variables Plant[f,w], Harvest[f,w]

### acres → used in capacity and labor constraints:
### sum_f Plant[f,w] * acres[f] ≤ capacity[w]

### Weather & regional extensions (optional later)
### region + (centroid_lat, centroid_lon) let you:
### join to regional weather series,
### model different planting windows by region, etc.

### Visualization
### You can color Gantt bars or maps by region.

In [35]:

# 

### 1️⃣ Understand what’s in the NOAA files
### From your files:
### noaa_il_daily_raw.csv
### Columns:
### station – station ID (e.g. GHCND:US1ILBN0014)

### date – "YYYY-MM-DD"
### PRCP – daily precipitation (inches)
### AWND – average daily wind speed (mph)
### TAVG – avg temperature (°F)
### TMAX – max temp (°F)
### TMIN – min temp (°F)
### noaa_il_weekly_agg.csv

### Columns:

### year – int
### week – ISO week number (1–52)
### PRCP – average daily precip over that week (inches/day)
### TMAX, TMIN, TAVG – average of daily values
### AWND – average wind
### We’ll turn the weekly file into a capacity table for the MILP.

## 2️⃣ Clean daily file (mostly for completeness / possible EDA)

In [36]:
# ImRkvVoWZZfIyRovfripXzAkOfNhzUol

In [37]:
daily_path = Path("/home/mak/Documents/Optimization/Project/data/raw/noaa_il_daily_raw.csv")
daily = pd.read_csv(daily_path)

In [38]:
daily.head(30)
# daily.shape
# daily['PRCP'].value_counts()

,station,date,AWND,PRCP,TAVG,TMAX,TMIN
0,GHCND:USW00094846,2017-01-01,4.9,0.00,27.0,40.0,17.0
1,GHCND:USW00094846,2017-01-02,6.3,0.11,34.0,40.0,25.0
2,GHCND:USW00094846,2017-01-03,10.1,0.00,38.0,39.0,19.0
3,GHCND:USW00094846,2017-01-04,17.0,0.00,18.0,19.0,7.0
4,GHCND:USW00094846,2017-01-05,12.8,0.00,9.0,13.0,3.0
5,GHCND:USW00094846,2017-01-06,10.1,0.00,4.0,9.0,0.0
6,GHCND:USW00094846,2017-01-07,10.7,0.00,6.0,18.0,0.0
7,GHCND:USW00094846,2017-01-08,9.6,0.00,9.0,19.0,2.0
8,GHCND:USW00094846,2017-01-09,13.0,0.02,22.0,33.0,19.0
9,GHCND:USW00094846,2017-01-10,23.3,0.20,38.0,51.0,25.0


In [39]:
# assume df_daily has columns: ["TMAX", "TMIN", "TAVG", ...]
# define which rows you consider "bad" TAVG
mask_bad_tavg = daily["TAVG"].isna()

# (optionally also treat zeros as bad if you know they’re placeholders, not real temps)
# mask_bad_tavg = df_daily["TAVG"].isna() | (df_daily["TAVG"] == 0)

daily.loc[mask_bad_tavg, "TAVG"] = (
    daily.loc[mask_bad_tavg, ["TMAX", "TMIN"]].mean(axis=1)
)
daily['TAVG'].value_counts()

TAVG
 74.0    243
 73.0    241
 71.0    237
 70.0    237
 33.0    236
        ... 
 4.5       1
-22.0      1
 3.5       1
-17.0      1
-11.0      1
Name: count, Length: 198, dtype: int64

In [40]:
daily['TAVG'].isna().sum()

np.int64(0)

In [41]:
# Parse date
daily["date"] = pd.to_datetime(daily["date"])

In [42]:
# Ensure numeric
for col in ["PRCP", "AWND", "TAVG", "TMAX", "TMIN"]:
    daily[col] = pd.to_numeric(daily[col], errors="coerce")

In [43]:
# Optional: drop rows that are completely missing weather
daily = daily.dropna(subset=["PRCP", "TMAX", "TMIN"], how="all").reset_index(drop=True)

In [44]:
daily.head(3)
daily.shape

(14608, 7)

In [45]:
daily['date'] = pd.to_datetime(daily['date'])

daily['year'] = daily['date'].dt.year
daily['week'] = daily['date'].dt.isocalendar().week.astype(int)


## 3️⃣ Clean + enhance weekly file (this is the important one)
## We’ll:
## Make sure types are right
## Compute weekly precipitation (inches/week) from the average daily PRCP
## Add a capacity factor (0–1) that we’ll use as a multiplier on planter/harvester capacity
## Optionally add a simple rain category for plotting / analysis

In [46]:
weekly = (
    daily
    .groupby(["year", "week"], as_index=False)
    .agg(
        prcp_week_in=("PRCP", "sum"),
        TMAX=("TMAX", "mean"),
        TMIN=("TMIN", "mean"),
        TAVG=("TAVG", "mean"),
        AWND=("AWND", "mean"),
    )
)

weekly.to_csv('/home/mak/Documents/Optimization/Project/data/processed/noaa_il_weekly_agg.csv', index=False)

In [47]:
weekly_path = Path("/home/mak/Documents/Optimization/Project/data/raw/noaa_il_weekly_agg.csv")
weekly = pd.read_csv(weekly_path)

In [48]:
print("Raw weekly shape:", weekly.shape)
print(weekly.head(30))

Raw weekly shape: (568, 7)
    year  week      PRCP       TMAX       TMIN       TAVG       AWND
0   2015     1  0.162789  34.697368  17.697368  27.450000   9.190909
1   2015     2  0.040194  18.744361   0.556391  10.314286  11.903896
2   2015     3  0.021089  33.541353  13.803030  25.200000   8.815584
3   2015     4  0.018651  39.533835  26.900763  34.142857   6.849351
4   2015     5  0.114625  35.639098  21.293233  30.571429   9.894805
5   2015     6  0.039961  31.984962  11.167939  22.942857   9.623377
6   2015     7  0.001202  29.112782  11.353383  21.114286  11.131169
7   2015     8  0.024865  21.000000   4.037594  14.771429   9.672727
8   2015     9  0.045405  22.481203   0.052632  12.914286   8.059740
9   2015    10  0.026745  33.219697  11.878788  25.228571   9.261842
10  2015    11  0.027761  57.315789  32.120301  46.200000   6.635065
11  2015    12  0.007442  55.548872  33.969925  46.485714   9.328571
12  2015    13  0.085290  43.624060  26.902256  36.028571  10.823377
13  201

In [49]:
# ---- Basic type cleanup ----
weekly["year"] = weekly["year"].astype(int)
weekly["week"] = weekly["week"].astype(int)

for col in ["PRCP", "TMAX", "TMIN", "TAVG", "AWND"]:
    weekly[col] = pd.to_numeric(weekly[col], errors="coerce")

In [50]:
weekly["prcp_week_in"] = weekly["PRCP"] * 7
weekly["prcp_week_in"].head(3)

0    1.139524
1    0.281357
2    0.147626
Name: prcp_week_in, dtype: float64

In [51]:
print(weekly.head(3))
weekly.columns
weekly.shape

   year  week      PRCP       TMAX       TMIN       TAVG       AWND  \
0  2015     1  0.162789  34.697368  17.697368  27.450000   9.190909   
1  2015     2  0.040194  18.744361   0.556391  10.314286  11.903896   
2  2015     3  0.021089  33.541353  13.803030  25.200000   8.815584   

   prcp_week_in  
0      1.139524  
1      0.281357  
2      0.147626  


(568, 8)

In [52]:
# ---- Compute weekly precipitation (inches/week) ----
# PRCP in your file is average daily precip over that week,
# so approximate total weekly precip as PRCP * 7
# ---- Define a capacity_factor based on total weekly rain ----
def capacity_from_rain(prcp_week):
    """
    Simple heuristic:
      - 0.0   in: perfect dry week → 1.0 (100% capacity)
      - <0.5  in: light rain       → 0.9
      - <1.5  in: moderate rain    → 0.7
      - <3.0  in: heavy rain       → 0.4
      - >=3.0 in: very wet         → 0.2
    """
    if pd.isna(prcp_week):
        return 0.8  # neutral if missing
    if prcp_week == 0:
        return 1.0
    if prcp_week < 0.5:
        return 0.9
    if prcp_week < 1.5:
        return 0.7
    if prcp_week < 3.0:
        return 0.4
    return 0.2

In [53]:
weekly["capacity_factor"] = weekly["prcp_week_in"].apply(capacity_from_rain)

# ---- Optional: rain category for plotting ----
def rain_bucket(prcp_week):
    if pd.isna(prcp_week):
        return "missing"
    if prcp_week == 0:
        return "dry"
    if prcp_week < 0.5:
        return "light"
    if prcp_week < 1.5:
        return "moderate"
    if prcp_week < 3.0:
        return "heavy"
    return "very_heavy"

weekly["rain_category"] = weekly["prcp_week_in"].apply(rain_bucket)

In [54]:
weekly["rain_category"].value_counts()

rain_category
moderate      247
light         245
heavy          62
very_heavy      9
dry             5
Name: count, dtype: int64

In [55]:
# ---- Keep only the columns we actually need for modeling ----
weekly_clean = weekly[
    ["year", "week",
     "prcp_week_in", "TMAX", "TMIN", "TAVG", "AWND",
     "capacity_factor", "rain_category"]
].sort_values(["year", "week"]).reset_index(drop=True)

In [56]:
weekly_clean

,year,week,prcp_week_in,TMAX,TMIN,TAVG,AWND,capacity_factor,rain_category
0,2015,1,1.139524,34.697368,17.697368,27.450000,9.190909,0.7,moderate
1,2015,2,0.281357,18.744361,0.556391,10.314286,11.903896,0.9,light
2,2015,3,0.147626,33.541353,13.803030,25.200000,8.815584,0.9,light
3,2015,4,0.130556,39.533835,26.900763,34.142857,6.849351,0.9,light
4,2015,5,0.802372,35.639098,21.293233,30.571429,9.894805,0.7,moderate
...,...,...,...,...,...,...,...,...,...
563,2025,40,0.003360,87.619048,57.746032,NaN,NaN,0.9,light
564,2025,41,0.640615,74.104000,48.056000,NaN,NaN,0.7,moderate
565,2025,42,1.029344,74.950820,53.892562,NaN,NaN,0.7,moderate
566,2025,43,0.110526,60.656000,37.896000,NaN,NaN,0.9,light


### 4️⃣ How this NOAA weekly table will be used later
### In the MILP (Gurobi)
### When we define weekly capacity constraints, we’ll do something like:
# Example parameters (you’ll define them when we build the model)
### base_planter_capacity = 600  # acres/week
### base_harvester_capacity = 800  # acres/week
# For each week w:
### eff_planter_cap[w]  = base_planter_capacity  * capacity_factor[w]
### eff_harvester_cap[w] = base_harvester_capacity * capacity_factor[w]
# Then constraints:
### sum_f Plant[f,w]   * area[f] <= eff_planter_cap[w]
### sum_f Harvest[f,w] * area[f] <= eff_harvester_cap[w]
# So rainier weeks literally shrink capacity in the optimization model.
### In Monte Carlo
### We’ll fit distributions to prcp_week_in (or to capacity_factor) over all years.
### For each simulation, we’ll draw a random capacity_factor[w] trajectory for the season.
### Then re-run the MILP under that scenario.
### In Forecasting
### Weather features for the ML model:
### prcp_week_in
### TAVG, TMAX, TMIN
### maybe lagged versions (last week’s rain, temp, etc.)
# Targets:
### % planted / % harvested for that week.
### If this looks good, NOAA is done ✅
### Next we can clean the labor dataset (no_of_worker2.csv), which will let us add labor-hour constraints to the MILP.

In [57]:
# ---- Clean Value: remove commas and convert to int ----
# df["Value"] = df["Value"].astype(str).str.replace(",", "")
# df["Value"] = pd.to_numeric(df["Value"], errors="coerce")

## 🔧 4. Convert workers → weekly labor hours

## We assume:
## 40 hours/week per worker
## Opportunity to allocate more hours in peak weeks

## 🔥 6. Convert yearly → weekly labor availability

## Since NASS/NOAA data is week-based, we create a weekly labor table.
## We need a simple distribution strategy:

## Option A (simple):
## Labor available evenly across all weeks.
## Option B (better for realism):

## Planting-heavy (weeks 16–22): 120% of average
## Harvest-heavy (weeks 35–44): 150% of average
## Other weeks: 75% of average

## Let’s implement Option B.

In [58]:
# ---- REALISTIC LABOR MODEL (REPLACES USDA LOGIC) ----

TOTAL_ACRES = fields_clean["acres"].sum()            # ~1700–2100 acres
HOURS_PER_ACRE = 3.5                                 # realistic corn labor hrs per acre per year
TOTAL_LABOR_HOURS_YEAR = TOTAL_ACRES * HOURS_PER_ACRE

def seasonal_labor_fraction(w):
    if 16 <= w <= 20:       # planting weeks
        return 0.04
    elif 36 <= w <= 45:     # harvest weeks
        return 0.06
    else:
        return 0.0054

labor_weekly = []
for year in range(2017, 2025):
    for week in range(1, 53):
        labor_weekly.append({
            "year": year,
            "week": week,
            "labor_hours": TOTAL_LABOR_HOURS_YEAR * seasonal_labor_fraction(week)
        })

weekly_labor = pd.DataFrame(labor_weekly)

In [59]:
weekly_labor.head(3)

,year,week,labor_hours
0,2017,1,38.072093
1,2017,2,38.072093
2,2017,3,38.072093


In [60]:
weekly_labor.head(3)

,year,week,labor_hours
0,2017,1,38.072093
1,2017,2,38.072093
2,2017,3,38.072093


## 📌 7. How labor dataset integrates into your pipeline
## ✔ Optimization (Gurobi)

## This provides the weekly constraint:

## sum_f Plant[f,w] * area[f] * labor_per_acre ≤ labor_hours[w]


## and similar for Harvest.
## ✔ Monte Carlo Simulation
## Labor availability is perturbed:
## labor_hours_sim[w] = labor_hours[w] * uniform(0.85, 1.15)
## ✔ Forecasting
## Use labor as a predictive feature:
## total labor available
## planting-week labor intensity
## ✔ Visualization
## Labor heatmap across weeks.

## 🧩 The Master Table Builder

In [61]:
# ---- 1. Merge weather + labor on year/week ----
weekly_master = weekly_clean.merge(
    weekly_labor,
    on=["year", "week"],
    how="left"
)

In [62]:
# ---- 2. Attach planting windows (per year) ----
weekly_master = weekly_master.merge(
    plantingWindows_df.rename(columns={"Year": "year"}),
    on="year",
    how="left"
)

In [63]:
# ---- 3. Attach harvest windows (per year) ----
weekly_master = weekly_master.merge(
    harvest_windows_df.rename(columns={"Year": "year"}),
    on="year",
    how="left"
)

In [64]:
# === FIX: Expand harvest window realistically ===
weekly_master["harvest_start_week"] = 32
weekly_master["harvest_end_week"]   = 50


In [65]:
# ---- 4. Create window flags ----
weekly_master["is_plant_window"] = (
    (weekly_master["week"] >= weekly_master["plant_start_week"]) &
    (weekly_master["week"] <= weekly_master["plant_end_week"])
)

In [66]:
weekly_master["is_harvest_window"] = (
    (weekly_master["week"] >= weekly_master["harvest_start_week"]) &
    (weekly_master["week"] <= weekly_master["harvest_end_week"])
)


In [67]:
# ---- 5. Sort & inspect ----
weekly_master = weekly_master.sort_values(["year", "week"]).reset_index(drop=True)
weekly_master.head()

,year,week,prcp_week_in,TMAX,TMIN,TAVG,AWND,capacity_factor,rain_category,labor_hours,plant_start_week,plant_end_week,harvest_start_week,harvest_end_week,is_plant_window,is_harvest_window
0,2015,1,1.139524,34.697368,17.697368,27.450000,9.190909,0.7,moderate,NaN,16,20,32,50,False,False
1,2015,2,0.281357,18.744361,0.556391,10.314286,11.903896,0.9,light,NaN,16,20,32,50,False,False
2,2015,3,0.147626,33.541353,13.803030,25.200000,8.815584,0.9,light,NaN,16,20,32,50,False,False
3,2015,4,0.130556,39.533835,26.900763,34.142857,6.849351,0.9,light,NaN,16,20,32,50,False,False
4,2015,5,0.802372,35.639098,21.293233,30.571429,9.894805,0.7,moderate,NaN,16,20,32,50,False,False


## This is correct because:
## ✔ Weeks 1–15 → not planting
## You set:
plant_start_week = 16
plant_end_week = 20

So week 1 → is_plant_window = False.

✔ Weeks 1–35 → not harvest

You set:

harvest_start_week = 36

harvest_end_week = 43

So week 1 → is_harvest_window = False.

✔ Rain category + capacity factor work

Week 1 has ~1.14 inches of rain → moderate rain → capacity factor = 0.7.
Correct.

✔ Labor hours = 0

Because your labor dataset likely only has 2017 & 2022 data.
Other years defaulted to 0 (we can fix this if needed, see below).

Everything is working.

⚠️ 2. Important fix: labor_hours = 0 for all years except census years

Right now:

USDA labor census only exists for 2017 and 2022

All other years show 0 labor-hours

For the MILP, you want non-zero labor for all weeks.

You have 2 good options:

Option A — Forward-fill from closest census year

This is realistic.

Example:

Years 2015, 2016 → use 2017 labor numbers

Years 2018, 2019, 2020, 2021 → use 2022

Years after 2022 → keep 2022

In [68]:
weekly_master['year'].value_counts()

year
2015    53
2016    53
2021    53
2020    53
2017    52
2019    52
2018    52
2022    52
2023    52
2024    52
2025    44
Name: count, dtype: int64

In [69]:
from pathlib import Path

# Create directories if not exist
Path("data/processed").mkdir(parents=True, exist_ok=True)
Path("data/raw").mkdir(parents=True, exist_ok=True)

# Save processed datasets
save_map = {
    "nass_corn_planting_weekly_clean.csv": df_planting_clean,
    "nass_corn_planting_windows.csv": plantingWindows_df,
    "nass_corn_harvest_weekly_clean.csv": df_harvested_clean,
    "nass_corn_harvest_windows.csv": harvest_windows_df,
    "illinois_corn_fields_clean.csv": fields_clean,
    "noaa_il_daily_clean.csv": daily,
    "noaa_il_weekly_clean.csv": weekly_clean,
    "labor_weekly_capacity_clean.csv": weekly_labor,
    "master_weekly_table_2017_2024.csv": weekly_master
}


for filename, df in save_map.items():
    df.to_csv(f"data/processed/{filename}", index=False)
    print(f"Saved: data/processed/{filename}")

Saved: data/processed/nass_corn_planting_weekly_clean.csv
Saved: data/processed/nass_corn_planting_windows.csv
Saved: data/processed/nass_corn_harvest_weekly_clean.csv
Saved: data/processed/nass_corn_harvest_windows.csv
Saved: data/processed/illinois_corn_fields_clean.csv
Saved: data/processed/noaa_il_daily_clean.csv
Saved: data/processed/noaa_il_weekly_clean.csv
Saved: data/processed/labor_weekly_capacity_clean.csv
Saved: data/processed/master_weekly_table_2017_2024.csv


In [70]:
df_harvested_clean.head(3)

,Year,week,week_ending,pct_harvested
0,2005,36,2005-09-11,6
1,2005,37,2005-09-18,13
2,2005,38,2005-09-25,30


In [71]:
df_planting_clean.head(3)

,Year,week,week_ending,pct_planted
194,2005,15,2005-04-17,35.0
195,2005,16,2005-04-24,64.0
196,2005,17,2005-05-01,82.0


In [72]:
# print(harvest_windows_df[harvest_windows_df["Year"] == 2017])


# 1️⃣ Make labor realistic (seasonal, not 5M hours every week)
Right now, after you build weekly_master, every row has the same labor_hours ≈ 5,057,910.
We’ll keep that as the annual pool, but scale it by week.

In [73]:
# wm = pd.read_csv("data/processed/master_weekly_table.csv")
# wm = wm[(wm["year"] >= 2017) & (wm["year"] <= 2024)].copy()
# wm.to_csv("data/processed/master_weekly_table_2017_2024.csv", index=False)

In [74]:
weekly_master.to_csv("data/processed/master_weekly_table_2017_2024.csv", index=False)


In [75]:
weekly_master.head(3)

,year,week,prcp_week_in,TMAX,TMIN,TAVG,AWND,capacity_factor,rain_category,labor_hours,plant_start_week,plant_end_week,harvest_start_week,harvest_end_week,is_plant_window,is_harvest_window
0,2015,1,1.139524,34.697368,17.697368,27.450000,9.190909,0.7,moderate,NaN,16,20,32,50,False,False
1,2015,2,0.281357,18.744361,0.556391,10.314286,11.903896,0.9,light,NaN,16,20,32,50,False,False
2,2015,3,0.147626,33.541353,13.803030,25.200000,8.815584,0.9,light,NaN,16,20,32,50,False,False


In [78]:
weekly_master[weekly_master["year"] == 2017][["week", "harvest_start_week", "harvest_end_week", "is_harvest_window"]]


,week,harvest_start_week,harvest_end_week,is_harvest_window
106,1,32,50,False
107,2,32,50,False
108,3,32,50,False
109,4,32,50,False
110,5,32,50,False
111,6,32,50,False
112,7,32,50,False
113,8,32,50,False
114,9,32,50,False
115,10,32,50,False


In [77]:
print(weekly_master[weekly_master["year"]==2017][["week","is_plant_window"]])


     week  is_plant_window
106     1            False
107     2            False
108     3            False
109     4            False
110     5            False
111     6            False
112     7            False
113     8            False
114     9            False
115    10            False
116    11            False
117    12            False
118    13            False
119    14            False
120    15             True
121    16             True
122    17             True
123    18             True
124    19             True
125    20             True
126    21             True
127    22            False
128    23            False
129    24            False
130    25            False
131    26            False
132    27            False
133    28            False
134    29            False
135    30            False
136    31            False
137    32            False
138    33            False
139    34            False
140    35            False
141    36            False
1